<a href="https://colab.research.google.com/github/valerio-unifei/ECAA07/blob/main/ECAA07_06_CRUD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CRUD Rest API

## Instalando bibliotecas

In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn

## Criando banco de dados

In [ ]:
import sqlite3
import pandas as pd

banco = 'ecaa07_crud.db'
conn = sqlite3.connect(banco)

c = conn.cursor()

# criando tabela de alunos
c.execute('''
CREATE TABLE IF NOT EXISTS alunos (
  aluno_id INTEGER PRIMARY KEY AUTOINCREMENT,
  aluno varchar(255) NOT NULL,
  matricula INT
);
''')

# criando tabela de notas
c.execute('''
CREATE TABLE IF NOT EXISTS notas (
  nota_id INTEGER PRIMARY KEY AUTOINCREMENT,
  aluno_id INTEGER REFERENCES alunos(aluno_id) ON DELETE CASCADE,
  nota REAL NOT NULL
);
''')

# visualizando tabelas criadas
print('Tabela alunos:')
display(pd.read_sql_query('SELECT * FROM alunos', con = conn))
print('Tabela notas:')
display(pd.read_sql_query('SELECT * FROM notas', con = conn))

conn.close()

## Inicializando Aplicação Rest API

In [28]:
from fastapi import FastAPI # cria o serviço de Rest API
from fastapi.responses import JSONResponse # converte valores para JSON

app = FastAPI()

@app.get('/')
def home():
  return 'ECAA07 Web Rest API'

### Listando Tabelas

In [29]:
@app.get('/alunos')
def alunos():
  conn = sqlite3.connect(banco)
  c = conn.cursor()
  c.execute('SELECT * FROM alunos ORDER BY matricula')
  tabela = c.fetchall() # retorna registros da consulta
  conn.close()
  return JSONResponse(tabela)

@app.get('/notas')
def notas():
  conn = sqlite3.connect(banco)
  c = conn.cursor()
  c.execute('SELECT n.nota_id, a.aluno_id, a.aluno, a.matricula, n.nota FROM alunos as a INNER JOIN notas as n ON a.aluno_id = n.aluno_id ORDER BY a.matricula')
  tabela = c.fetchall() # retorna registros da consulta
  conn.close()
  return JSONResponse(tabela)

### Inserindo Alunos e Notas

In [30]:
@app.post('/notas/')
def insert_notas(aluno:str, matricula:int, notas:str):
  conn = sqlite3.connect(banco)
  c = conn.cursor()
  c.execute('INSERT INTO alunos (aluno, matricula) VALUES (?,?)',[aluno, matricula])
  aluno_id = c.lastrowid # última chave primária criada
  print('aluno_id =',aluno_id)

  linhas = 0
  for nota in notas.split():
    nota_int = int(nota)
    c.execute('INSERT INTO notas (aluno_id, nota) VALUES (?,?)',[aluno_id, nota_int])
    linhas += c.rowcount # registro modificados no SQL
  
  conn.commit()
  conn.close()
  print('linhas =',linhas)

  if linhas > 0:
    return 'Aluno e notas inseridas'
  else:
    return 'Problema na inserção de notas'

### Atualizando e Removendo Notas

In [31]:
@app.delete('/notas/remove/{nota_id}')
def delete_notas(nota_id:int):
  conn = sqlite3.connect(banco)
  c = conn.cursor()
  c.execute('DELETE FROM notas WHERE nota_id = ?',[nota_id])
  apagados = c.rowcount # registro alterados no SQL

  conn.commit()
  conn.close()

  if apagados > 0:
    return 'Nota Removida'
  else:
    return 'Nota não encontrada'

@app.post('/notas/atualiza/')
def update_notas(nota_id:int, nota:int):
  conn = sqlite3.connect(banco)
  c = conn.cursor()
  c.execute('UPDATE notas SET nota = ? WHERE nota_id = ?',[nota, nota_id])
  atualizados = c.rowcount # registro alterados no SQL

  conn.commit()
  conn.close()

  if atualizados > 0:
    return 'Nota Atualizada'
  else:
    return 'Nota não encontrada'

## Ativando serviço Rest API

Para acessar o serviço vá no link que aparecerá na execução:

*Abra no navegador: http://###-###-###.ngrok.io/docs*

**Este bloco de código não irá parar de executar, pois senão o serviço cairá.**

Presssione o PLAY (>) novamente para parar.

In [ ]:
import nest_asyncio #permite execução de servidor em loop no Colab
from pyngrok import ngrok #dá um nome de DNS real para a aplicação
import uvicorn #gestor de aplicação web 

ngrok_tunnel = ngrok.connect(8000)
print()
print('Abra no navegador:', ngrok_tunnel.public_url + '/docs')
print()
nest_asyncio.apply()
uvicorn.run(app, port=8000)